<a href="https://colab.research.google.com/github/Madhav2008/Project140/blob/main/Project140.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop


 61% 5.00M/8.20M [00:01<00:00, 3.51MB/s]
100% 8.20M/8.20M [00:01<00:00, 5.66MB/s]


In [ ]:
!unzip articles-sharing-reading-from-cit-deskdrop

Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: shared_articles.csv     
  inflating: users_interactions.csv  


In [ ]:
import pandas as pd 
import csv

df1 = pd.read_csv('shared_articles.csv')
df2 = pd.read_csv('users_interactions.csv')

data1 = df1['eventType'].tolist()

for index,data in enumerate(data1):
  if data == 'CONTENT REMOVED':
    del data1[index]
 
df1.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [ ]:
data2 = df2['eventType'].tolist()

Views = 0
Followers = 0
Likes = 0
BookMarks = 0
Comments = 0

for index, data in enumerate(data2):
  if data == 'VIEW':
    Views += 1

  elif data == 'LIKE':
    Likes += 1

  elif data == 'BOOKMARK':
    BookMarks += 1

  elif data == 'FOLLOW':
    Followers += 1

  elif data == 'COMMENT CREATED':
    Comments += 1

print(f'Likes: {Likes}, Views: {Views}, BookMarks: {BookMarks}, Followers: {Followers}, Comments Created: {Comments}')

Likes: 5745, Views: 61086, BookMarks: 2463, Followers: 1407, Comments Created: 1611


In [ ]:
import statistics

wData = [Likes, Views, BookMarks, Followers, Comments]

C = statistics.mean(wData)
print(C)

14462.4


In [ ]:
import numpy as np

m = np.quantile(wData, 0.9)
print(m)

38949.600000000006


In [ ]:
q_articles = df1.copy()
print(q_articles.shape)

(3122, 13)


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = wData[0] + wData[1] + wData[2] + wData[3] + wData[4]
    R = C
    return (v/(v+m) * R) + (m/(m+v) * C)

q_articles['score'] = q_articles.apply(weighted_rating, axis=1)

In [ ]:
q_articles = q_articles.sort_values('score', ascending=False)
q_articles.head(20)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang,score
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en,14462.4
2097,1471525780,CONTENT SHARED,7286613907013366088,-6067316262393890508,-2873231259470527751,NaN,NaN,NaN,HTML,http://www.iii.org/issue-update/self-driving-c...,Self-Driving Cars and Insurance,THE TOPIC Each new generation of cars is equip...,en,14462.4
2075,1471350108,CONTENT SHARED,-5328971462978089293,-5639303336519418445,-1067585767182117848,NaN,NaN,NaN,HTML,http://exame.abril.com.br/marketing/noticias/p...,P&G cancela verba para anúncios segmentados no...,No mesmo dia em que o Facebook confirmou uma a...,pt,14462.4
2076,1471352269,CONTENT SHARED,1349971382589894814,3829784524040647339,-6107808233631554003,NaN,NaN,NaN,HTML,https://medium.com/@nntaled/the-most-intoleran...,The Most Intolerant Wins: The Dictatorship of ...,The Most Intolerant Wins: The Dictatorship of ...,en,14462.4
2077,1471352498,CONTENT SHARED,-2584174137395076448,534764222466712491,-2603418925334381846,NaN,NaN,NaN,HTML,http://epoca.globo.com/vida/noticia/2016/08/el...,"Elektro, Google e Sama lideram entre as melhor...","Sama, primeiro lugar entre as empresas médias ...",pt,14462.4
2078,1471353094,CONTENT SHARED,-7585525446062170144,-1032019229384696495,880025729283447629,NaN,NaN,NaN,HTML,https://techcrunch.com/2016/08/15/google-duo/,"Google launches Duo video calling app, a dull ...",Can Google spring back from its social failure...,en,14462.4
2079,1471353504,CONTENT SHARED,2388097612905924826,3829784524040647339,-6107808233631554003,NaN,NaN,NaN,HTML,http://www.wired.com/2016/08/open-source-won-now/,"Open Source Won. So, Now What?",The government is now a little more open. This...,en,14462.4
2080,1471361066,CONTENT SHARED,-2561023317421962555,-1387464358334758758,3366208659216606452,NaN,NaN,NaN,HTML,http://googlediscovery.com/2016/08/15/google-v...,Google vai encerrar Hangouts On Air para refor...,"A partir do dia 12 de setembro, a Hangouts On ...",pt,14462.4
2081,1471362160,CONTENT SHARED,7207307032404768106,-4627026983118548639,2463901734176270737,NaN,NaN,NaN,HTML,http://blogs.gartner.com/chris-ross/2016/08/09...,Gartner Hype Cycle for Digital Marketing and A...,"Hype, by definition should make us all a littl...",en,14462.4
2082,1471364061,CONTENT SHARED,3609315114795782005,3829784524040647339,-6107808233631554003,NaN,NaN,NaN,HTML,https://www.whitehouse.gov/webform/rfi-prepari...,Request for Information: Preparing for the Fut...,Request for Information: Preparing for the Fut...,en,14462.4


In [ ]:
import plotly.express as px

fig = px.bar((q_articles.head(10).sort_values('score', ascending=True)), x="score", y="eventType", orientation='h')
fig.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(q_articles['title'])

print(q_articles['title'])

0       Ethereum, a Virtual Currency, Enables Transact...
2097                      Self-Driving Cars and Insurance
2075    P&G cancela verba para anúncios segmentados no...
2076    The Most Intolerant Wins: The Dictatorship of ...
2077    Elektro, Google e Sama lideram entre as melhor...
                              ...                        
1043    Confira as novidades apresentadas no Google I/...
1044    São Paulo recebe maior evento nacional voltado...
1045                               Sem Parar vs Conectcar
1046                   Sem Parar vs Conectcar - DanBP.org
3121                 The 2017 Acquia Partners of the Year
Name: title, Length: 3122, dtype: object


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
q_articles = q_articles.reset_index()
indices = pd.Series(q_articles.index, index=q_articles['title'])

In [ ]:
def get_recommendations(title, cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    article_indices = [i[0] for i in sim_scores]
    return q_articles['title'].iloc[article_indices]

In [ ]:
get_recommendations("Self-Driving Cars and Insurance", cosine_sim2)

1550    All Tesla cars built from now on have everythi...
47      Uber's first self-driving cars will start pick...
3007    Intel buys computer vision startup Itseez to i...
1214    Not Your Father's Insurance Industry - Insuran...
1489    IoT Insurance: Trends in Home, Life & Auto Ins...
2334                          The insurance tech equation
1750                        The new-world insurance agent
53                  The new, scary face of auto insurance
234        Making digital strategy a reality in insurance
351     Blockchain Huge Potential For The Insurance In...
Name: title, dtype: object

In [ ]:
df1.to_csv('article.csv')
files.download('article.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>